In [ ]:
!pip install --user pytube
!pip install --user icalendar
!pip install --user qgrid

In [1]:
from pytube import YouTube, Playlist
import pandas as pd
from datetime import datetime, timedelta
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download("punkt")
import math
from icalendar import Calendar, Event
from ipywidgets import interact
import qgrid

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def get_runtime_and_title(url):
    if 'playlist' in url:
        playlist = Playlist(url)
        total_runtime = 0
        video_count = 0
        for video in playlist.videos:
            total_runtime += video.length
            video_count += 1

        video_data = {"Title":playlist.title,"Run Time": convert_seconds_to_hms(total_runtime), "Video Count": video_count,"Video Link": url}

    else:
        video = YouTube(url)
        title = video.title
        runtime = video.length
        video_data = [{"Title": title, "Run Time": convert_seconds_to_hms(runtime),"Video Count":1,"Video Link": url}]
   
    return video_data

def create_video_data(playlist_url):
    
    playlist = Playlist(playlist_url)
    # Create an empty DataFrame to store the video data
    df = pd.DataFrame(columns=['Video Title', 'Run Time', 'Video Link', 'Keywords'])

    # Iterate through the videos in the playlist
    for video in playlist.videos:
        # Extract video title, run time, and video link
        video_title = video.title
        video_duration = convert_seconds_to_hms(video.length)
        video_description = video.description

        # Extract keywords from the video title and description
        title_keywords = extract_keywords(video_title)# Extract keywords from the video description (if available)
         # Extract keywords from the video.keywords (if available)
        video_keywords = video.keywords
        if video_keywords:
            keyword_string = ", ".join(video_keywords)
        else:
            keyword_string = ""

        all_keywords = title_keywords + video_keywords
        keywords_string = ", ".join(all_keywords)

        # Append video information and keywords to the DataFrame
        df = df.append({'Video Title': video_title, 'Run Time': video_duration, 'Video Link': video.watch_url, 'Keywords': keywords_string}, ignore_index=True)
        
    return df

def convert_seconds_to_hms(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        
# Define a function to extract keywords from text
def extract_keywords(text):
    # Use regular expressions to remove special characters, symbols, and numbers
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize the cleaned text
    words = word_tokenize(cleaned_text)

    # Create a list of English stopwords and additional common words to exclude
    custom_stopwords = set(stopwords.words("english") + ["for", "or", "and", "the", "is", "are", "it", "in", "on"])
    
    # Remove punctuation, stopwords and common words and also lowercase the words
    words = [word.lower() for word in words if word.isalpha()]
    words = [word for word in words if word not in custom_stopwords]
    
    return words

def create_learning_group(df,allocated_time_in_min,sort_by_runtime):

    # Create a 'Group' column in the DataFrame
    df[['SortKey','Group']] = 0,''
    # Convert the 'Run Time' column to timedelta
    df['Run Time'] = pd.to_timedelta(df['Run Time'])
    if sort_by_runtime:
        df = df.sort_values(by='Run Time')
    # Initialize variables
    total_time = timedelta()
    part_number = 1
    start_index = 1

    # Create an empty list to store the rows to append
    rows_to_append = []

    # Iterate through the DataFrame and add rows
    for index, row in df.iterrows():
        if row['Run Time'] > timedelta(minutes=allocated_time_in_min):
            # Calculate the number of parts needed
            num_parts = float(row['Run Time'].total_seconds() / (allocated_time_in_min * 60))
            # Always spend 20% more time than the dedicated one for the video

            for part in range(int(math.ceil(num_parts*1.3))):
                new_row = row.copy()
                new_row['SortKey'] = part_number
                rows_to_append.append(new_row)
                part_number += 1
            # Remove the original row from df
            df = df.drop(index)
        else:
            if total_time + row['Run Time'] <= timedelta(minutes=allocated_time_in_min):
                total_time += row['Run Time']
                df.at[index, 'SortKey'] = part_number
            else:
                total_time = row['Run Time']
                part_number += 1
                df.at[index, 'SortKey'] = part_number

    # Append the duplicated rows to the DataFrame
    df = pd.concat([df, pd.DataFrame(rows_to_append)], ignore_index=True)

    # Sort the DataFrame by the 'SortKey' column
    df = df.sort_values(by='SortKey')
    df['SortKey'] = df['SortKey'].diff().eq(1).cumsum() + df['SortKey'].iat[0]
    df['Group'] = 'Part - ' + df['SortKey'].astype(str)
    # Reset the index
    df.reset_index(drop=True, inplace=True)
    # Drop the temporary 'SortKey' column
    df = df.drop(columns='SortKey')

    return df

In [3]:
# Define the URL of the YouTube playlist
# List of playlist URLs
playlist_urls = ["https://www.youtube.com/playlist?list=PLe0U7sHuld_qIILgg-2ESRCPWu-WBalFJ",
"https://www.youtube.com/playlist?list=PLAeu18HndGgBR-QLw8b8Wzp0gLiVfCS7n",
"https://www.youtube.com/playlist?list=PLe0U7sHuld_pZllkKAojENaQLYeOWfED7",
"https://www.youtube.com/playlist?list=PLOlK8ytA0MgjYGVrz0hS4w3UPQ1-VV2uX",
"https://www.youtube.com/playlist?list=PLqnslRFeH2UqLwzS0AwKDKLrpYBKzLBy2",
"https://www.youtube.com/playlist?list=PL7yh-TELLS1FuqLSjl5bgiQIEH25VEmIc",
"https://www.youtube.com/playlist?list=PLUOa-qvvZolCoiF8CuqCyVU9tG2v8cjE6",
"https://www.youtube.com/playlist?list=PLUkh9m2BorqnKWu0g5ZUps_CbQ-JGtbI9",
"https://www.youtube.com/playlist?list=PLAeu18HndGgBR-QLw8b8Wzp0gLiVfCS7n",
"https://www.youtube.com/playlist?list=PLAeu18HndGgDAWJOAPaqARiMkMJ1u-EOm",
"https://www.youtube.com/playlist?list=PLAeu18HndGgD-btpZ7rb358WGAHH1-ZcU",
"https://www.youtube.com/playlist?list=PLAeu18HndGgB-KWCMyZCKCgZbpik2I9A3"]

In [4]:
video_details =list(map(get_runtime_and_title, playlist_urls))
video_details = pd.DataFrame(video_details)
video_details = video_details.assign(Start_Date=None, Start_Time=None, Time_to_Spend=None)
video_details

,Title,Run Time,Video Count,Video Link,Start_Date,Start_Time,Time_to_Spend
0,hg,99:04:08,42,https://www.youtube.com/playlist?list=PLe0U7sH...,None,None,None
1,DUTCH FOR BEGINNERS COURSE (A0-A1),04:35:19,28,https://www.youtube.com/playlist?list=PLAeu18H...,None,None,None
2,Dutch,01:05:34,4,https://www.youtube.com/playlist?list=PLe0U7sH...,None,None,None
3,AWS DATA ENGINEERING,01:58:35,18,https://www.youtube.com/playlist?list=PLOlK8yt...,None,None,None
4,Advanced Python - Complete Course,06:00:02,21,https://www.youtube.com/playlist?list=PLqnslRF...,None,None,None
5,Python Advanced Tutorials,01:50:16,10,https://www.youtube.com/playlist?list=PL7yh-TE...,None,None,None
6,All episodes: newest first,11:20:57,140,https://www.youtube.com/playlist?list=PLUOa-qv...,None,None,None
7,Valuation,06:03:34,25,https://www.youtube.com/playlist?list=PLUkh9m2...,None,None,None
8,DUTCH FOR BEGINNERS COURSE (A0-A1),04:35:19,28,https://www.youtube.com/playlist?list=PLAeu18H...,None,None,None
9,Dutch beginners & intermediates mix (A0-A2),06:56:32,36,https://www.youtube.com/playlist?list=PLAeu18H...,None,None,None


In [7]:
# Display the DataFrame as an interactive table
qgrid.enable()
qgrid_widget = qgrid.show_grid(video_details, grid_options={'editable': True})
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
# Create empty columns for Start Date, Start Time, and Time to Spend
video_details['Start Date'] = ""
video_details['Start Time'] = ""
video_details['Time to Spend'] = ""

# Iterate through each row to get user input
for index, row in video_details.iterrows():
    print(f"Row {index + 1}: {row['Title']}")
    start_date = input("Enter start date (YYYY-MM-DD): ")
    start_time = input("Enter start time (HH:MM:SS): ")
    time_to_spend = input("Enter time to spend (HH:MM:SS): ")
    
    video_details.at[index, 'Start Date'] = start_date
    video_details.at[index, 'Start Time'] = start_time
    video_details.at[index, 'Time to Spend'] = time_to_spend

In [ ]:
# Display the resulting DataFrame
video_details

In [ ]:
video_details.to_csv("Plant to Learn.csv", index=False)

In [ ]:
video_details = pd.read_csv("Plant to Learn.csv")

In [ ]:
video_details.sort_values(by='Start Date')

In [ ]:
# Initialize the iCalendar
cal = Calendar()

for _,ROW in video_details.iterrows():
    
    # Define a start date and time
    start_date = pd.to_datetime(ROW['Start Date'] + ' ' + ROW['Start Time']) 
    
    playlist_data = create_video_data(ROW['Video Link'])
    if ROW['Title'] == 'hg':
        sort_by_runtime= True
    else:
        sort_by_runtime=False
        
    df = create_learning_group(playlist_data,int(ROW['Time to Spend']),sort_by_runtime)

    # Group the DataFrame by 'Category'
    df_dict = dict(iter(df.groupby('Group')))
    
    for group, dft in df_dict.items():
    # Calculate event duration from Run Time
        duration = timedelta(minutes=int(ROW['Time to Spend']))

        # Skip weekends (Saturday and Sunday)
        while start_date.weekday() >= 5:  # 5 and 6 correspond to Saturday and Sunday
            start_date += timedelta(days=1)

        # Create an event for each row
        event = Event()
        event.add('summary',ROW['Title']+ group)
        event.add('dtstart', start_date)
        event.add('dtend', start_date + duration)

        # Combine Video Link and Description in the description field
        event.add('description', dft)

        cal.add_component(event)

        # Increment start date for the next event
        start_date += timedelta(days=1)


In [ ]:
# Write the calendar to a file
with open('outlook_calendar.ics', 'wb') as f:
    f.write(cal.to_ical())

In [ ]:
df = create_video_data("https://www.youtube.com/playlist?list=PLe0U7sHuld_qIILgg-2ESRCPWu-WBalFJ")

In [ ]:
sort=False
playlist_data = create_video_data("https://www.youtube.com/playlist?list=PLAeu18HndGgBR-QLw8b8Wzp0gLiVfCS7n")
df = create_learning_group(playlist_data,45,sort)
df

In [ ]:
df1 = create_learning_group(df,45)
df1.to_csv("df1.csv")

In [ ]:
# Create a Pandas Excel writer using pandas ExcelWriter
with pd.ExcelWriter('playlists.xlsx') as writer:
    for sheet_name, dataframe in playlist_data.items():
        # Clean and shorten the sheet name to 31 characters
        cleaned_sheet_name = " ".join(extract_keywords(sheet_name))[:31]
        dataframe.to_excel(writer, sheet_name=cleaned_sheet_name, index=False)

In [ ]:

playlist_data 

In [ ]:

df.head(10)

In [ ]:
df['Group']

In [ ]:
for _, row in df.iterrows():
    # Calculate event duration from Run Time
    duration = timedelta(minutes=45)
    
    # Skip weekends (Saturday and Sunday)
    while start_date.weekday() >= 5:  # 5 and 6 correspond to Saturday and Sunday
        start_date += timedelta(days=1)

    # Create an event for each row
    event = Event()
    event.add('summary', row['Video Title'])
    event.add('dtstart', start_date)
    event.add('dtend', start_date + duration)
    
    # Combine Video Link and Description in the description field
    event.add('description', f'{row["Keywords"]} - Video Link: {row["Video Link"]}')
    
     # Set the event color to orange
    event.add('color', 'orange')

    cal.add_component(event)

    # Increment start date for the next event
    start_date += timedelta(days=1)

